# 5.2 张量与自动求导

PyTorch使用中有几个比较重要的概念，如张量（Tensor）、自动求导（Autograd）等，需要初学者理解，并灵活使用。

## 5.2.1 张量

（1）初始化张量

张量可以用Python的list数据进行初始化，并自动判断其数据类型，也可以使用NumPy数组进行初始化。张量和NumPy数组也可以互相转化。例子如下：

In [1]:
import torch  # 导入PyTorch
import numpy as np  # 导入NumPy

data = [[1, 2],[3, 4]]
np_array = np.array(data)  # 用list数组创建Numpy对象
t = torch.tensor(data)  # 用list数组创建Tensor对象
tensor_from_np = torch.from_numpy(np_array)  # 用NumPy数组创建Tensor对象

np_from_tensor = t.numpy()  # 将Tensor对象转换为NumPy数组
listdata = tensor_from_np.tolist()  # 将Tensor对象转换为list数组
print(t, '\n', np_from_tensor, '\n', listdata)

tensor([[1, 2],
        [3, 4]]) 
 [[1 2]
 [3 4]] 
 [[1, 2], [3, 4]]


也可以从另一个张量对象复制数据来初始化新的张量，例如：

In [2]:
tensor_data = t.clone()  # 返回与t值相同的tensor,新对象存储在新的内存中
new_data = t.detach()  # 返回与t完全相同的tensor,新对象与旧对象t共享内存
ones_data = torch.ones_like(t)  # 和t形状一致的全1张量
zeros_data = torch.zeros_like(t)  # 和t形状一致的全0张量
rand_data = torch.rand_like(t, dtype=torch.float)  # 和t形状一致的随机浮点数张量

（2）张量的属性

张量的属性描述了其形状、数据类型、存储设备及在内存中的存储形式等信息。张量的形状表示各个维度的大小。张量支持的数据类型主要是不同精度的浮点数和整数，如32bit的float类型、64bit的double类型等。存储设备是指张量在CPU上计算，还是在GPU上计算。张量在内存中的存储形式表示是按稠密矩阵存储，还是按稀疏矩阵存储。例如：

In [3]:
tensor = torch.rand(3,4)  # 3x4的随机tensor
print(f"张量形状: {tensor.shape}")
print(f"张量数据类型: {tensor.dtype}")
print(f"张量存储设备: {tensor.device}")

张量形状: torch.Size([3, 4])
张量数据类型: torch.float32
张量存储设备: cpu


Tensor也可存储的GPU上进行加速运算，例如：

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu" #判断是否可用CUDA加速运算
print(device)
tensor = tensor.to(device) #将Tensor放到GPU上
print(f"张量存储设备: {tensor.device}")

cpu
张量存储设备: cpu


（3）索引和切片

张量的索引和切片类似NumPy的索引和切片。示例如下：

In [5]:
tensor = torch.randint(1,100,(4, 4))  #4行4列，各元素值为[1，100)区间的随机数
print(f"第一行: {tensor[0]}")
print(f"第一列: {tensor[:, 0]}")
print(f"最后一列: {tensor[..., -1]}")  # ...和:是一样的
tensor[:, 1] = 0  # 第二列置为0
print(tensor)

第一行: tensor([51, 83, 82, 20])
第一列: tensor([51, 55, 44, 66])
最后一列: tensor([20, 23, 28, 27])
tensor([[51,  0, 82, 20],
        [55,  0, 16, 23],
        [44,  0, 40, 28],
        [66,  0, 55, 27]])


（4）张量连接

多个张量可以在某个维度上进行连接，除了要连接的维度值不同，其他维度的值应保持一致，否则不可以进行连接。例如两个张量的形状分别为[2, 4]和[3, 4]，则可以在第一个维度，即纵向上进行连接。示例如下：

In [6]:
tensor1 = torch.randint(1,100,(4, 4))
tensor2 = torch.rand(3,4) 
tensor3 = torch.rand(4,2) 
t1 = torch.cat([tensor1, tensor2], dim=0)  # 纵向连接
t2 = torch.cat((tensor1, tensor3), dim=1)  # 横向连接,用()和[]均可
print(tensor1.shape, tensor2.shape, tensor3.shape, t1.shape, t2.shape)

torch.Size([4, 4]) torch.Size([3, 4]) torch.Size([4, 2]) torch.Size([7, 4]) torch.Size([4, 6])


torch.cat函数返回张量维度不会变，只是连接维度上的值变为各个被链接的张量在该维度的值之和。除了torch.cat函数外，torch.stack函数也可以进行连接，不过各个被链接的张量的各个维度值都应一致，返回的新张量维度会多增加一维。示例如下：

In [7]:
tensor1 = torch.randint(1,100,(2, 4))
tensor2 = torch.zeros(2,4) 
tensor3 = torch.ones(2,4) 
t1 = torch.stack([tensor1, tensor2, tensor3],dim=0)
t2 = torch.stack((tensor1, tensor2, tensor3),dim=1)
t3 = torch.stack((tensor1, tensor2, tensor3),dim=2)
print(tensor1.shape, tensor2.shape, tensor3.shape, t1.shape, t2.shape, t3.shape)
t1,t2,t3

torch.Size([2, 4]) torch.Size([2, 4]) torch.Size([2, 4]) torch.Size([3, 2, 4]) torch.Size([2, 3, 4]) torch.Size([2, 4, 3])


(tensor([[[ 5., 79., 23., 43.],
          [30., 27., 23., 83.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.]]]),
 tensor([[[ 5., 79., 23., 43.],
          [ 0.,  0.,  0.,  0.],
          [ 1.,  1.,  1.,  1.]],
 
         [[30., 27., 23., 83.],
          [ 0.,  0.,  0.,  0.],
          [ 1.,  1.,  1.,  1.]]]),
 tensor([[[ 5.,  0.,  1.],
          [79.,  0.,  1.],
          [23.,  0.,  1.],
          [43.,  0.,  1.]],
 
         [[30.,  0.,  1.],
          [27.,  0.,  1.],
          [23.,  0.,  1.],
          [83.,  0.,  1.]]]))

（5）数学运算

张量作为矩阵，可以进行加、减、乘、转置、按元素乘、按元素除等操作。示例如下：

In [8]:
tensor1 = torch.randint(1,100,(2, 4))
tensor2 = torch.ones(2,4) 
t_add1 = tensor1 + tensor2  # 张量加法，本质上是按元素相加
t_add2 = tensor1.add(tensor2)  # 与上面的操作是一致的
t_add3 = torch.add(tensor1, tensor2)  # 与上面的操作是一致的

t_add4 = tensor1 + 3  # 张量所有元素都加3，得到新的张量，原张量未改变
t_add5 = tensor1.add(3)  # 与上面的操作是一致的
t_add6 = torch.add(tensor1, 3)  # 与上面的操作是一致的

t_add7 = tensor1.add_(3)  # 张量所有元素都加3，原张量tensor1也被修改。

上面的例子中实现了张量的加操作。张量的各种基本运算和例中的加操作类型，都可以有多种操作方法。通常操作函数，如add，后加“_”表示执行完运算后，原张量也被修改。下面给出一些示例：

In [9]:
t_sub1 = tensor1 - tensor2  # 张量减法
t_sub2 = tensor1.sub(tensor2)  # 与上面的操作是一致的
t_sub4 = 1 - tensor2  # 张量所有元素都被1减
t_sub6 = torch.sub(tensor1, 1)  # 张量所有元素都减1

#张量乘法，参与运算的张量类型需一致
t_matmul1 = tensor1.float() @ tensor2.T  # T属性表示张量转置,按单精度浮点数(32bit)进行相乘
t_matmul2 = tensor1.matmul(tensor2.T.long())  # 与上面的操作不同，两张量按长整型(64bit)进行相乘

#按元素相乘，参加运算的张量形状应一致
t_mul1 = tensor1 * tensor2  
t_mul3 = torch.mul(tensor1, tensor2)  # 与上面的操作是一致的
t_mul4 = tensor1 * 3  # 张量每元素都乘以3
t_mul5 = tensor1.mul(3)  # 与上面的操作是一致的

#按元素相除，参加运算的张量形状应一致
t_div1 = tensor1 / tensor1  
t_div2 = tensor1.div(tensor2) 
t_div3 = torch.div(tensor1, tensor2)  # 与上面的操作是一致的

我们在实际应用中，会经常使用一维或二维张量的乘法操作，PyTorch也提供了一些常用的特殊运算函数，如dot函数、mm函数等，示例如下。

In [10]:
t1=torch.randn((5))  # 一维张量
t2=torch.ones((5))  # 一维张量
t3=torch.randn((2,5))  # 二维张量
t4=torch.ones((5,2))  # 二维张量
t_d1 = torch.dot(t1, t2)  # dot函数仅支持两个一维向量的点集
t_d2 = torch.matmul(t1, t2)  # 与上面的操作是一致的
print(t_d1 == t_d2)  # 观察两个结果是否相同

t_m1 = torch.mm(t3, t4)  # mm函数仅支持两个二维张量的相乘
t_m2 = torch.matmul(t3, t4)  # 与上面的操作一致
print(t_m1 == t_m2)  # 观察两个结果是否相同

t_n1 = torch.matmul(t3, t1) # 二维张量和一维张量相乘，一维张量自动维度扩展, 结果会删掉扩展维度
print(t_n1.shape)  # 打印计算结果形状
t_n2 = torch.matmul(t3, t1.view(5,1)).T # 手动扩展进行计算，与上面的操作结果一致
print(t_n1 == t_n2)  # 观察两个结果是否相同

tensor(True)
tensor([[True, True],
        [True, True]])
torch.Size([2])
tensor([[True, True]])


若要取张量中的某个元素，变为普通的数据类型进行运算，可使用item()函数。例如：

In [11]:
tensor = torch.randperm(10)
sum = tensor.sum()
sum_item = sum.item()
print(tensor)
print(sum, type(sum))
print(sum_item, type(sum_item))

tensor([7, 9, 3, 0, 1, 8, 6, 2, 4, 5])
tensor(45) <class 'torch.Tensor'>
45 <class 'int'>


（6）张量的其他运算

PyTorch提供的张量运算函数是非常丰富的，如幂运算、指数运算（以e为底数）、对数运算、近似运算（如取整）、统计运算（如取平均值）等等，在此不再给出示例，只列举一些常用的函数，读者可自行练习这些函数的使用方法。

## 5.2.2 Autograd自动求导

神经网络模型训练时，最常用的是BP算法，即反向传播算法。模型参数根据梯度进行学习，为了计算梯度，PyTorch提供了Autograd支持自动求导来计算梯度。
下面以最简单的一层神经网络为例，给出Autograd进行反向传播优化参数的示例。例子中，设x和y是真实数据，z为模型预测值，设z = w * x + b，则w和b是模型需要优化的参数，通过loss损失可根据梯度下降法来优化w和b。


In [16]:
import torch
from torch.nn.functional import binary_cross_entropy_with_logits
x = torch.ones(5)  # 输入x设为[1.,1.,1.,1.,1.]
y = torch.zeros(3)  # 输出y设为[0.,0.,0.]
w = torch.randn(5, 3, requires_grad=True)  # w形状为[5, 3]，梯度计算设为True
b = torch.randn(3, requires_grad=True)  # b形状为[3, 1]，梯度计算设为True
z = torch.matmul(x, w) + b  # z = w * x + b
loss = binary_cross_entropy_with_logits(z, y)  # 计算损失根据loss来优化网络参数
loss.backward()  # 损失反向传播进行自动求导，得到参数梯度
print(w.grad)  # 输出w的梯度，存储在w的grad属性中，grad属性也是张量
print(b.grad)  # 输出b的梯度

tensor([[0.3267, 0.3003, 0.0754],
        [0.3267, 0.3003, 0.0754],
        [0.3267, 0.3003, 0.0754],
        [0.3267, 0.3003, 0.0754],
        [0.3267, 0.3003, 0.0754]])
tensor([0.3267, 0.3003, 0.0754])


上例中，得到了w和b的梯度后，就可以通过设置学习率来修正w和b的值了，实际上修正的方法PyTorch也封装好了，后续的章节中将给出具体的示例。

默认情况下，所有张量的requires_grad属性被设置为True，表示都在跟踪梯度历史，但是有些情况下并不需要跟踪梯度，比如测试集上的评估就不需要反向传播来跟踪梯度，可用以下代码来停止对计算结果的梯度跟踪


In [18]:
z = torch.matmul(x, w)+b  # z = w * x + b
print(z.requires_grad)
with torch.no_grad():  # 禁用梯度跟踪
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


禁用梯度跟踪也可以使用下面的方法：

In [19]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


后续的实战中，读者将会发现，在进行深度模型训练时，一般都需要调用损失张量的backward()函数进行自动求导。